# Categorical Boost

In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import sklearn
from Functions import prepro
pd.set_option("display.max_columns", 400)
pd.set_option("display.max_rows", 400)

In [2]:
train_df = pd.read_csv("../../data/Processed/train2.csv", index_col='id')
test_df = pd.read_csv("../../data/Processed/test2.csv", index_col="id")
print(train_df.shape)
print(test_df.shape)

(132250, 191)
(28340, 190)


In [ ]:
#train_df.head()

In [ ]:
#train_df.info()

In [ ]:
#test_df.head()

### 武器の情報を個人ごとからチームごとに変える
category1, 2を落とす、

他のカテゴリをチームごとにする

reskinそのまま

In [ ]:
# 落とさないほうが精度上がってしまった。。。

"""suffixes = ["-A1", "-A2", "-A3", "-A4", "-B1", "-B2", "-B3", "-B4"]
drop_cols = ["category1", "category2"]

for c in drop_cols:
    for s in suffixes:
        col = c+s
        train_df.drop(col, axis=1, inplace=True)
        test_df.drop(col, axis=1, inplace=True)
"""

In [ ]:
#train_df.columns

In [3]:
y = train_df["y"]
train_df = train_df.drop("y", axis=1)

In [4]:
# add team info
train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2"])

(160590, 190)
special-A
special-B
subweapon-A
subweapon-B
category1-A
category1-B
category2-A
category2-B
(160590, 278)
complete


In [5]:
# add disconnection info

train_df = prepro.add_disconnection(train_df)
test_df = prepro.add_disconnection(test_df)

In [6]:
# add info about numeric column
from Functions import prepro

num_cols = ["level", "range", "rapid", "atack"]

train_df = prepro.flat(train_df, num_cols)
test_df = prepro.flat(test_df, num_cols)

train_df = prepro.add_numeric_info(train_df, num_cols)
test_df = prepro.add_numeric_info(test_df, num_cols)

In [7]:
# make input

drop_cols = ["id", "lobby", "lobby-mode",  "period", "game-ver", "A1-weapon", "A2-weapon", "A3-weapon", "A4-weapon", \
              "B1-weapon", "B2-weapon", "B3-weapon", "B4-weapon"]
X, test_X = prepro.make_input(train_df, test_df, drop_cols, categorical_encode = False, verbose=False)

In [ ]:
"""print(train_df.shape)
print(test_df.shape)
for col in X.columns:
    print(col)"""

In [14]:
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import Pool, CatBoostClassifier

categorical_features_indices = np.where(X.dtypes != np.float)[0]

random.seed(0)

In [15]:
# 全データを5つに分割
SIZE = X.shape[0]
K = 5

def make_kfolds(SIZE, K):
    # return list object, each element is indices of its fold
    FOLD_SIZE = int(SIZE/K)
    res = []
    indices = [i for i in range(SIZE)]
    for i in range(K-1):
        fold = random.sample(indices, FOLD_SIZE)
        indices = list(set(indices) - set(fold))
        res.append(fold)
    res.append(indices)
    return res

folds = make_kfolds(SIZE, K)
print(len(folds))
for i, fold in enumerate(folds):
    print("fold ", i+1, " size is ", len(fold))
    

    
if SIZE != len(set(sum(folds, []))):
    print("error is occuring in spliting")
else :
    print("successfully split")


5
fold  1  size is  26450
fold  2  size is  26450
fold  3  size is  26450
fold  4  size is  26450
fold  5  size is  26450
successfully split


In [27]:
params = {
    "loss_function" : "Logloss",
    "eval_metric" : "Logloss",
    "iterations":200,
    "learning_rate" : 0.1,
    "use_best_model": True,

    

}

THRESHOLD = 0.50
models = []
cv_scores = []
temp = 0
train_pred = []

all_indices = sum(folds, [])
for i in range(K):
    valid_indices = folds[i]
    train_indices = list(set(all_indices) - set(valid_indices))
    # print("train ", len(train_indices), " , valid ", len(valid_indices))
    
    train_X = X.iloc[train_indices]
    train_y = y.iloc[train_indices]
    valid_X = X.iloc[valid_indices]
    valid_y = y.iloc[valid_indices]
    
    train_data = Pool(train_X, train_y, cat_features=categorical_features_indices)
    valid_data = Pool(valid_X, valid_y, cat_features=categorical_features_indices)
    
    model = CatBoostClassifier(**params)

    
    model.fit(
        train_data,
        eval_set=valid_data,
        early_stopping_rounds=10,
        verbose=10,
        use_best_model=True,
    )
    
    pred = model.predict(valid_X)
    # pred = np.where(pred < THRESHOLD, 0, 1)
    train_pred.append(pred)
    temp += np.sum(pred)
    
    score = accuracy_score(pred, valid_y)
    
    models.append(model)
    cv_scores.append(score)
    
    
print("cv score : ", np.mean(cv_scores))    
print("cv ratio : ", temp / SIZE)

0:	learn: 0.6923983	test: 0.6923906	best: 0.6923906 (0)	total: 778ms	remaining: 2m 34s
10:	learn: 0.6881603	test: 0.6884269	best: 0.6884269 (10)	total: 10.4s	remaining: 2m 59s
20:	learn: 0.6859922	test: 0.6869619	best: 0.6869619 (20)	total: 20.1s	remaining: 2m 51s
30:	learn: 0.6842954	test: 0.6858608	best: 0.6858608 (30)	total: 30s	remaining: 2m 43s
40:	learn: 0.6831518	test: 0.6850963	best: 0.6850963 (40)	total: 40.2s	remaining: 2m 35s
50:	learn: 0.6821503	test: 0.6845987	best: 0.6845987 (50)	total: 51.7s	remaining: 2m 30s
60:	learn: 0.6810374	test: 0.6840780	best: 0.6840773 (59)	total: 1m 2s	remaining: 2m 22s
70:	learn: 0.6800230	test: 0.6835352	best: 0.6835352 (70)	total: 1m 11s	remaining: 2m 10s
80:	learn: 0.6792773	test: 0.6832978	best: 0.6832877 (79)	total: 1m 20s	remaining: 1m 58s
90:	learn: 0.6785028	test: 0.6830766	best: 0.6830766 (90)	total: 1m 34s	remaining: 1m 53s
100:	learn: 0.6777354	test: 0.6828883	best: 0.6828834 (99)	total: 1m 49s	remaining: 1m 47s
110:	learn: 0.676792

70:	learn: 0.6799235	test: 0.6838629	best: 0.6838629 (70)	total: 55.6s	remaining: 1m 41s
80:	learn: 0.6792787	test: 0.6835849	best: 0.6835849 (80)	total: 1m 4s	remaining: 1m 34s
90:	learn: 0.6784112	test: 0.6833047	best: 0.6833047 (90)	total: 1m 11s	remaining: 1m 26s
100:	learn: 0.6775622	test: 0.6830272	best: 0.6830272 (100)	total: 1m 20s	remaining: 1m 18s
110:	learn: 0.6767541	test: 0.6828336	best: 0.6828336 (110)	total: 1m 28s	remaining: 1m 10s
120:	learn: 0.6758893	test: 0.6826519	best: 0.6826511 (119)	total: 1m 36s	remaining: 1m 3s
130:	learn: 0.6749062	test: 0.6824180	best: 0.6824180 (130)	total: 1m 44s	remaining: 55.2s
140:	learn: 0.6738077	test: 0.6822220	best: 0.6822220 (140)	total: 1m 52s	remaining: 47.1s
150:	learn: 0.6728436	test: 0.6820186	best: 0.6820186 (150)	total: 2m	remaining: 39s
160:	learn: 0.6720503	test: 0.6819182	best: 0.6819182 (160)	total: 2m 8s	remaining: 31.1s
170:	learn: 0.6710553	test: 0.6817232	best: 0.6817232 (170)	total: 2m 16s	remaining: 23.1s
180:	lear

In [28]:
preds = []
for i in range(K):
    model = models[i]
    pred = model.predict(test_X)
    preds.append(pred)
    print(np.sum(pred) / pred.shape[0])
    
preds = np.array(preds)
preds = np.mean(preds, axis=0)
print(np.sum(preds) / preds.shape[0])


temp = pd.DataFrame({"pred":pred})
# temp.hist(bins=20)

0.5764290755116444
0.5608327452364149
0.5840860973888496
0.5841919548341566
0.5803458009880028
0.5771771347918138


In [31]:
preds = np.where(preds < THRESHOLD, 0, 1)
print(np.sum(preds) / preds.shape[0])

submit_df = pd.DataFrame({'y': preds})
submit_df.index.name = 'id'
submit_df.to_csv('../Submissions/submission_cat_1.csv')

0.5811926605504587


In [24]:
importance = pd.DataFrame(models[1].feature_importance(), index=X.columns, columns=['importance']).sort_values('importance')
display(importance)

AttributeError: 'CatBoostClassifier' object has no attribute 'feature_importance'

In [32]:
submit_df

,y
id,
0,0
1,0
2,1
3,1
4,1
...,...
28335,1
28336,1
28337,1
